# Ensemble of ensembles - model stacking

* **Ensemble with different types of classifiers**: 
  * Different types of classifiers (E.g., logistic regression, decision trees, random forest, etc.) are fitted on the same training data
  * Results are combined based on either 
    * majority voting (classification) or 
    * average (regression)
  

* **Ensemble with a single type of classifier**: 
  * Bootstrap samples are drawn from training data 
  * With each bootstrap sample, model (E.g., Individual model may be decision trees, random forest, etc.) will be fitted 
  * All the results are combined to create an ensemble. 
  * Suitabe for highly flexible models that is prone to overfitting / high variance. 

***

## Combining Method

* **Majority voting or average**: 
  * Classification: Largest number of votes (mode) 
  * Regression problems: Average (mean).
  
  
* **Method of application of meta-classifiers on outcomes**: 
  * Binary outcomes: 0 / 1 from individual classifiers
  * Meta-classifier is applied on top of the individual classifiers. 
  
  
* **Method of application of meta-classifiers on probabilities**: 
  * Probabilities are obtained from individual classifiers. 
  * Applying meta-classifier
  

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv("dataset\WA_Fn-UseC_-HR-Employee-Attrition.csv")
df.pop('EmployeeNumber')
df.pop('Over18')
df.pop('StandardHours')
df.pop('EmployeeCount')
y = df['Attrition']
X = df
X.pop('Attrition')
from sklearn import preprocessing
le = preprocessing.LabelBinarizer()
y = le.fit_transform(y)
ind_BusinessTravel = pd.get_dummies(df['BusinessTravel'], prefix='BusinessTravel')
ind_Department = pd.get_dummies(df['Department'], prefix='Department')
ind_EducationField = pd.get_dummies(df['EducationField'], prefix='EducationField')
ind_Gender = pd.get_dummies(df['Gender'], prefix='Gender')
ind_JobRole = pd.get_dummies(df['JobRole'], prefix='JobRole')
ind_MaritalStatus = pd.get_dummies(df['MaritalStatus'], prefix='MaritalStatus')
ind_OverTime = pd.get_dummies(df['OverTime'], prefix='OverTime')
df1 = pd.concat([ind_BusinessTravel, ind_Department, ind_EducationField, ind_Gender, 
                 ind_JobRole, ind_MaritalStatus, ind_OverTime])
df1 = pd.concat([ind_BusinessTravel, ind_Department, ind_EducationField, ind_Gender, 
                 ind_JobRole, ind_MaritalStatus, ind_OverTime, df.select_dtypes(['int64'])], axis=1)
df1.dropna(inplace=True)
df1.shape

C:\Users\VIPUL.GAUR\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(1470, 51)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1, y)

In [5]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [6]:
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    '''
    print the accuracy score, classification report and confusion matrix of classifier
    '''
    if train:
        '''
        training performance
        '''
        print("Train Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, clf.predict(X_train))))
        print("Classification Report: \n {}\n".format(classification_report(y_train, clf.predict(X_train))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, clf.predict(X_train))))

        res = cross_val_score(clf, X_train, y_train.ravel(), cv=10, scoring='accuracy')
        print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))
        
    elif train==False:
        '''
        test performance
        '''
        print("Test Result:\n")        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, clf.predict(X_test))))
        print("Classification Report: \n {}\n".format(classification_report(y_test, clf.predict(X_test))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, clf.predict(X_test))))    
        

## Model 1: Decision Tree

In [7]:
from sklearn.tree import DecisionTreeClassifier

In [8]:
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [9]:
print_score(tree_clf, X_train, y_train, X_test, y_test, train=True)
print_score(tree_clf, X_train, y_train, X_test, y_test, train=False)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       929
           1       1.00      1.00      1.00       173

    accuracy                           1.00      1102
   macro avg       1.00      1.00      1.00      1102
weighted avg       1.00      1.00      1.00      1102


Confusion Matrix: 
 [[929   0]
 [  0 173]]

Average Accuracy: 	 0.8085
Accuracy SD: 		 0.0368
Test Result:

accuracy score: 0.7935

Classification Report: 
               precision    recall  f1-score   support

           0       0.87      0.88      0.88       304
           1       0.40      0.36      0.38        64

    accuracy                           0.79       368
   macro avg       0.63      0.62      0.63       368
weighted avg       0.79      0.79      0.79       368


Confusion Matrix: 
 [[269  35]
 [ 41  23]]



## Model 2: Random Forest

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train.ravel())

C:\Users\VIPUL.GAUR\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [13]:
print_score(rf_clf, X_train, y_train, X_test, y_test, train=True)
print_score(rf_clf, X_train, y_train, X_test, y_test, train=False)

Train Result:

accuracy score: 0.9782

Classification Report: 
               precision    recall  f1-score   support

           0       0.97      1.00      0.99       929
           1       1.00      0.86      0.93       173

    accuracy                           0.98      1102
   macro avg       0.99      0.93      0.96      1102
weighted avg       0.98      0.98      0.98      1102


Confusion Matrix: 
 [[929   0]
 [ 24 149]]

Average Accuracy: 	 0.8657
Accuracy SD: 		 0.0169
Test Result:

accuracy score: 0.8342

Classification Report: 
               precision    recall  f1-score   support

           0       0.84      0.99      0.91       304
           1       0.64      0.11      0.19        64

    accuracy                           0.83       368
   macro avg       0.74      0.55      0.55       368
weighted avg       0.80      0.83      0.78       368


Confusion Matrix: 
 [[300   4]
 [ 57   7]]



In [17]:
en_en = pd.DataFrame()

In [18]:
tree_clf.predict_proba(X_train)

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [19]:
tree_clf.predict_proba?

In [20]:
en_en['tree_clf'] = pd.DataFrame(tree_clf.predict_proba(X_train))[1]
en_en['rf_clf'] =  pd.DataFrame(rf_clf.predict_proba(X_train))[1]
col_name = en_en.columns
en_en = pd.concat([en_en, pd.DataFrame(y_train).reset_index(drop=True)], axis=1)

In [21]:
en_en.head()

tree_clf  rf_clf  0
0       1.0     1.0  1
1       0.0     0.0  0
2       0.0     0.0  0
3       1.0     0.5  1
4       0.0     0.0  0

In [22]:
tmp = list(col_name)
tmp.append('ind')
en_en.columns = tmp

In [23]:
en_en.head()

tree_clf  rf_clf  ind
0       1.0     1.0    1
1       0.0     0.0    0
2       0.0     0.0    0
3       1.0     0.5    1
4       0.0     0.0    0

In [24]:
en_en.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1102 entries, 0 to 1101
Data columns (total 3 columns):
tree_clf    1102 non-null float64
rf_clf      1102 non-null float64
ind         1102 non-null int32
dtypes: float64(2), int32(1)
memory usage: 21.6 KB


# Meta Classifier

In [32]:
from sklearn.linear_model import LogisticRegression

In [33]:
m_clf = LogisticRegression(fit_intercept=False)

In [34]:
m_clf.fit(en_en[['tree_clf', 'rf_clf']], en_en['ind'])

C:\Users\VIPUL.GAUR\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [35]:
en_test = pd.DataFrame()

In [36]:
en_test['tree_clf'] = pd.DataFrame(tree_clf.predict_proba(X_test))[1]
en_test['rf_clf'] =  pd.DataFrame(rf_clf.predict_proba(X_test))[1]
col_name = en_en.columns
en_test['combined'] = m_clf.predict(en_test[['tree_clf', 'rf_clf']])

In [37]:
col_name = en_test.columns
tmp = list(col_name)
tmp.append('ind')

In [38]:
tmp

['tree_clf', 'rf_clf', 'combined', 'ind']

In [39]:
en_test = pd.concat([en_test, pd.DataFrame(y_test).reset_index(drop=True)], axis=1)

In [40]:
en_test.head()

tree_clf  rf_clf  combined  0
0       1.0     0.3         1  0
1       0.0     0.2         0  1
2       0.0     0.1         0  0
3       0.0     0.6         0  1
4       0.0     0.1         0  0

In [27]:
en_test.columns = tmp

In [28]:
print(pd.crosstab(en_test['ind'], en_test['combined']))

combined    0   1
ind              
0         257  55
1          35  21


In [29]:
print(round(accuracy_score(en_test['ind'], en_test['combined']), 4))

0.7554


In [30]:
print(classification_report(en_test['ind'], en_test['combined']))

              precision    recall  f1-score   support

           0       0.88      0.82      0.85       312
           1       0.28      0.38      0.32        56

    accuracy                           0.76       368
   macro avg       0.58      0.60      0.58       368
weighted avg       0.79      0.76      0.77       368



***